In [1]:
import torch
import torch.nn as nn

Gather information fusing components consist of two parts.
First is the element representation matrix $\textbf{E}$, which could be seen as a static representation of all the element and it is shared with all users. This is an advantage, as we could use it also for sparse data and we do not need any additional attributes for the users.
Second is the compact representation of element w.r.t $u_i$ we constructed in the previous step, $\mathbb{Z}_i = \{ z_{i,1},...,z_{i,|\mathcal{V}_i|} \}$.

We use $E_i$ to denote the hidden state of user $i$, which is initializes as $E$. The most recent state $E^{update}_{i, I(j)}$ is achieved by updating the user state $E_i$ iteratively as follows:,
$E^{update}_{i, I(j)} = (1- \beta_{i,I(j)} \cdot \gamma_{I(j)}) \cdot E_{i,I(j)} + (\beta_{i,I(j)} \cdot \gamma_{I(j)}) \cdot z_{i,j}$, where $I(\cdot)$ is a function that maps element $v_{i,j}$ to its corresponding index in $E_i, \beta_{i,j}$ and $\gamma_j$ are the j-th dimention of $\beta_i$ and $\gamma$.

In [ ]:
class global_gated_update(nn.Module):

    def __init__(self, items_total, item_embedding):
        super(global_gated_update, self).__init__()
        
        # TODO: ali items_total predstavlja vse možne izdelke (za vse houdeholde, kot je v E), ali samo za trenutni houdehold
        # Mislim, da so to VSI IZDELKI!!
        # items_total: vsi možni izdelki (za vse householde)
        self.items_total = items_total
        
        # item_embedding: matrika E (statična matrika), v kateri so neke reprezentacije za vse možne izdelke (na začetku inicializirana)
        self.item_embedding = item_embedding

        # Uteži za updejtanje
        self.gamma = nn.Parameter(torch.rand(items_total, 1), requires_grad=True)

    def forward(self, graph, nodes, nodes_output):
        """
        :param graph: batched graphs, with the total number of nodes is `node_num`,
                        including `batch_size` disconnected subgraphs
        :param nodes: tensor (n_1+n_2+..., )
        :param nodes_output: the output of self-attention model in time dimension, (n_1+n_2+..., F)
        :return:
        """
        
        nums_nodes, id = graph.batch_num_nodes(), 0
        # .batch_num_nodes(): vrne število nodov za vsak graf v batchu
        # Npr seznam [število nodov v prvem grafu, število nodov v drugem grafu, ..., vseh grafov je toliko kot je košaric za]
        # A ni število nodov pri vseh grafih za isti household enako? A je tuki šel s to for zanko samo zaradi tega, ker je funkcija .batch_num_nodes() komot?
        
        # E_iI(j) = Iz matrike E vzame samo tiste node oz izdelke, ki se pojavijo pri pozameznem householdu
        # To je ista matrika kot E na začetku -- zakaj potem spreminjamo in ne vzamemo kar item_embedding? 
        items_embedding = self.item_embedding(torch.tensor([i for i in range(self.items_total)]) #.to(nodes.device))
        batch_embedding = []
                                              
        # Sprehodimo se čez vsak graf za posamezen houdehold 
        # Z eno for zanko se sprehodimo čez en graf (j od 1 do |vi|, okno velikosti |vi|)     -- (d:id + num_nodes): na tak način gremo čez vse izdelke v grafu
        # Se ubistvu sprehajamo po grafih                                     
        for num_nodes in nums_nodes:
                                              
            # TUKI NOTR JE VSE NA NIVOJU GRAFOV... extractamo izdelke iz trenutnega grafa v for zanki!                                  
                                              
            # tensor, shape, (user_nodes, item_embed_dim)                                      
            # output_node_features: z_ij (vzamemo vse take)                                  
            output_node_features = nodes_output[id:id + num_nodes, :]
            
            # Izdelki za trenuten graf v for zanki za trenuten householda                                 
            output_nodes = nodes[id: id + num_nodes]
                                              
            # beta, tensor, (items_total, 1), indicator vector, appear item -> 1, not appear -> 0
            # Najprej nastavimo same 0                                  
            beta = torch.zeros(self.items_total, 1).to(nodes.device)
            # Potem pa za tiste izdelki, ki se pojavijo, beto nastavimo na 1                                  
            beta[output_nodes] = 1
                                              
            # update global embedding by gated mechanism
            # broadcast (items_total, 1) * (items_total, item_embed_dim) -> (items_total, item_embed_dim)
            
            ### ----------------- UPDATE ENAČBA ----------------- ###                                  
            # To je prvi člen enačbe (5) za update     
            
            # Tukaj vzamemo matriko E za vse člene, zato da embed nastavimo na enake dimenzije (da jo potem res updejtamo)                                  
            embed = (1 - beta * self.gamma) * items_embedding.clone() 
            # appear items: (1 - self.gamma) * origin + self.gamma * update, not appear items: origin
            # beta * self.gamma : to je 0 za izdelke, ki se ne pojavijo, in utež za izdelke, ki se pojavijo
            # (1 - beta * self.gamma) : to je 1 (1 - 0) za izdelke, ki se ne pojavijo, in (1. utež) za izdelke, ki se pojavijo 
            # Če (1 - beta * self.gamma) množimo z matriko embeddingov za VSE izdelke, se potem tisti, ki so množeni z eni (tisti, ki se ne pojavijo) itak ne spremenijo...
                                              
                                              
            # Stanje spreminjamo samo pri izdelkih, ki se pojavijo (output_nodes), ostalega ne spreminjamo  
            # Zakaj v drugem členu tukaj nimamo bete? A zato, ker je množimo z vektorjem s samimi 1?  Moglo bi bit tkole, da bi blo prou po formuli
                    # (beta[output_nodes] + self.gamma[output_nodes]) * output_node_features                               
            embed[output_nodes, :] = embed[output_nodes, :] + self.gamma[output_nodes] * output_node_features
                                              
            batch_embedding.append(embed)
            
            # Se premaknemo na naslednji graf                                  
            id += num_nodes
                                              
        # (B, items_total, item_embed_dim)
        batch_embedding = torch.stack(batch_embedding)
                                              
        return batch_embedding